<a href="https://colab.research.google.com/github/Rishitha110506/IBS-02/blob/main/IBS_2_ENDSEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 32.8 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving alzheimers_dataset.fasta to alzheimers_dataset.fasta


In [ ]:
import os
print(os.listdir())


['.config', 'alzheimers_dataset.fasta', 'sample_data']


redundancy removed using cosine similarity:

“Sequence redundancy was removed using cosine similarity based on k-mer frequency vectorization. Sequences with similarity greater than 90% were filtered to retain non-redundant sequences.”

Bioinformatics standard approach


Numerical representation of sequences


Similarity-aware filtering




Better than exact duplicate removal

In [ ]:
from Bio import SeqIO, pairwise2
import numpy as np
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# =========================
# 1️⃣ Load FASTA file
# =========================

fasta_file = "alzheimers_dataset.fasta"

sequences = []
ids = []

for record in SeqIO.parse(fasta_file, "fasta"):
    sequences.append(str(record.seq))
    ids.append(record.id)

print("Total sequences BEFORE redundancy removal:", len(sequences))

# =========================
# 2️⃣ Convert sequences to k-mer vectors
# =========================

def kmer_vector(seq, k=3):
    kmers = [seq[i:i+k] for i in range(len(seq)-k+1)]
    counts = Counter(kmers)
    return counts

# Create vocabulary of all kmers
all_kmers = set()
kmer_counts = []

for seq in sequences:
    counts = kmer_vector(seq)
    kmer_counts.append(counts)
    all_kmers.update(counts.keys())

all_kmers = sorted(list(all_kmers))

# Convert to numeric matrix
matrix = []

for counts in kmer_counts:
    vector = [counts.get(kmer, 0) for kmer in all_kmers]
    matrix.append(vector)

matrix = np.array(matrix)

# =========================
# 3️⃣ Cosine similarity redundancy removal
# =========================

threshold = 0.90
keep_indices = []

for i in range(len(matrix)):
    keep = True
    for j in keep_indices:
        sim = cosine_similarity([matrix[i]], [matrix[j]])[0][0]
        if sim >= threshold:
            keep = False
            break
    if keep:
        keep_indices.append(i)

unique_sequences = [sequences[i] for i in keep_indices]

print("Total sequences AFTER cosine similarity filtering:", len(unique_sequences))

# =========================
# 4️⃣ Global alignment scores (first 20)
# =========================

print("\nGlobal Alignment Scores (first 20 pairs):")

for i in range(min(20, len(unique_sequences)-1)):
    score = pairwise2.align.globalxx(
        unique_sequences[i],
        unique_sequences[i+1],
        score_only=True
    )
    print(f"Seq{i} vs Seq{i+1} Score:", score)

# =========================
# 5️⃣ Sequence accuracy
# =========================

lengths = [len(seq) for seq in unique_sequences]
avg_length = np.mean(lengths)

correct = sum(1 for l in lengths if abs(l - avg_length) <= 50)
sequence_accuracy = correct / len(unique_sequences)

print("\nSequence Accuracy:", sequence_accuracy)

# =========================
# 6️⃣ Precision, Recall, F1, Confusion Matrix
# =========================

y_true = [1 if l > avg_length else 0 for l in lengths]
y_pred = [1 if abs(l - avg_length) <= 50 else 0 for l in lengths]

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)

print("\nPrecision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", cm)



Total sequences BEFORE redundancy removal: 1229
Total sequences AFTER cosine similarity filtering: 496

Global Alignment Scores (first 20 pairs):
Seq0 vs Seq1 Score: 304.0
Seq1 vs Seq2 Score: 25.0
Seq2 vs Seq3 Score: 31.0
Seq3 vs Seq4 Score: 17.0
Seq4 vs Seq5 Score: 20.0
Seq5 vs Seq6 Score: 13.0
Seq6 vs Seq7 Score: 17.0
Seq7 vs Seq8 Score: 63.0
Seq8 vs Seq9 Score: 38.0
Seq9 vs Seq10 Score: 9.0
Seq10 vs Seq11 Score: 29.0
Seq11 vs Seq12 Score: 42.0
Seq12 vs Seq13 Score: 28.0
Seq13 vs Seq14 Score: 6.0
Seq14 vs Seq15 Score: 7.0
Seq15 vs Seq16 Score: 23.0
Seq16 vs Seq17 Score: 65.0
Seq17 vs Seq18 Score: 180.0
Seq18 vs Seq19 Score: 128.0
Seq19 vs Seq20 Score: 263.0

Sequence Accuracy: 0.14516129032258066

Precision: 0.2361111111111111
Recall: 0.1223021582733813
F1 Score: 0.16113744075829384
Confusion Matrix:
 [[302  55]
 [122  17]]


Exact Duplicate Removal using Set:

We remove sequences that are exactly identical by converting the list of sequences into a Python set.

A set automatically keeps only unique values and removes repeated ones.

Removes only 100% identical sequences


 Does NOT remove similar sequences


 Does NOT check sequence similarity percentage

In [ ]:
from Bio import SeqIO
from Bio import pairwise2
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

# ==============================
# 1️⃣ LOAD FASTA FILE
# ==============================
file_path = "alzheimers_dataset.fasta"   # change if needed

sequences = [str(record.seq) for record in SeqIO.parse(file_path, "fasta")]

print("Total sequences loaded:", len(sequences))

# ==============================
# 2️⃣ REMOVE DUPLICATES
# ==============================
unique_sequences = list(set(sequences))

print("Sequences after removing duplicates:", len(unique_sequences))

# ==============================
# 3️⃣ REDUNDANCY INFO
# ==============================
duplicates_removed = len(sequences) - len(unique_sequences)

print("Duplicates removed:", duplicates_removed)

# ==============================
# 4️⃣ GLOBAL ALIGNMENT SCORES (FIRST 20)
# ==============================
print("\nGlobal Alignment Scores (first 20 unique sequences):")

match_score = 1
mismatch_penalty = -1
gap_penalty = -1

limit = min(20, len(unique_sequences))

for i in range(limit):
    for j in range(i + 1, limit):
        score = pairwise2.align.globalms(
            unique_sequences[i],
            unique_sequences[j],
            match_score,
            mismatch_penalty,
            gap_penalty,
            gap_penalty,
            score_only=True
        )
        print(f"Seq {i+1} vs Seq {j+1} Score = {score}")

# ==============================
# 5️⃣ SEQUENCE ACCURACY
# ==============================
# True labels → duplicates are positive
true_labels = [1 if seq in unique_sequences else 0 for seq in sequences]

# Predicted labels → model predicts duplicate if repeated
seen = set()
predicted_labels = []

for seq in sequences:
    if seq in seen:
        predicted_labels.append(1)
    else:
        predicted_labels.append(0)
        seen.add(seq)

accuracy = accuracy_score(true_labels, predicted_labels)

print("\nSequence Accuracy:", accuracy)

# ==============================
# 6️⃣ PRECISION, RECALL, F1
# ==============================
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

print("\nPrecision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# ==============================
# 7️⃣ CONFUSION MATRIX
# ==============================
cm = confusion_matrix(true_labels, predicted_labels)

print("\nConfusion Matrix:")
print(cm)


Total sequences loaded: 1229
Sequences after removing duplicates: 838
Duplicates removed: 391

Global Alignment Scores (first 20 unique sequences):
Seq 1 vs Seq 2 Score = 208.0
Seq 1 vs Seq 3 Score = -247.0
Seq 1 vs Seq 4 Score = -669.0
Seq 1 vs Seq 5 Score = -656.0
Seq 1 vs Seq 6 Score = -574.0
Seq 1 vs Seq 7 Score = -250.0
Seq 1 vs Seq 8 Score = -383.0
Seq 1 vs Seq 9 Score = -260.0
Seq 1 vs Seq 10 Score = 268.0
Seq 1 vs Seq 11 Score = -602.0
Seq 1 vs Seq 12 Score = -346.0
Seq 1 vs Seq 13 Score = -380.0
Seq 1 vs Seq 14 Score = -255.0
Seq 1 vs Seq 15 Score = 273.0
Seq 1 vs Seq 16 Score = -260.0
Seq 1 vs Seq 17 Score = 217.0
Seq 1 vs Seq 18 Score = -393.0
Seq 1 vs Seq 19 Score = -1546.0
Seq 1 vs Seq 20 Score = -270.0
Seq 2 vs Seq 3 Score = -259.0
Seq 2 vs Seq 4 Score = -665.0
Seq 2 vs Seq 5 Score = -653.0
Seq 2 vs Seq 6 Score = -560.0
Seq 2 vs Seq 7 Score = -269.0
Seq 2 vs Seq 8 Score = -402.0
Seq 2 vs Seq 9 Score = -262.0
Seq 2 vs Seq 10 Score = 206.0
Seq 2 vs Seq 11 Score = -593.0
Seq